In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, fftshift
import matplotlib.colors as colors

def grid_normaliser(source):
            if source == 'yes':
                freq_x = (np.fft.fftfreq(grid_size_x, d=grid_spacing_x))*(primitive_lattice_vector)
                freq_y = (np.fft.fftfreq(grid_size_y, d=grid_spacing_y))*(primitive_lattice_vector)
            else: 
                freq_x = (np.fft.fftfreq(grid_size_x, d=grid_spacing_x))*(2*np.pi)
                freq_y = (np.fft.fftfreq(grid_size_y, d=grid_spacing_y))*(2*np.pi)

            return(freq_x, freq_y)    

is_periodic = input("Is your structure periodic? (yes/no): ")

if is_periodic == "yes":
    primitive_lattice_vector = float(input("Primitive lattice vector in atomic units:"))
    cub_file1 = input("Enter the file path for your .cub file: ")

    grid_normaliser_result = grid_normaliser(is_periodic)
    freq_x, freq_y = grid_normaliser_result

    with open(cub_file1, 'r') as file1:
        data1_lines = file1.readlines()
    
        grid_size_x = int(data1_lines[3].split()[0])
        print(grid_size_x)
        grid_spacing_x = float(data1_lines[3].split()[1])
        print(grid_spacing_x)

        grid_size_y = int(data1_lines[4].split()[0])
        print(grid_size_y)
        grid_spacing_y = float(data1_lines[4].split()[2])
        print(grid_spacing_y)

        grid_size_z = int(data1_lines[5].split()[0])
        print(grid_size_z)
        grid_spacing_z = float(data1_lines[5].split()[3])
        print(grid_spacing_z)


        for i in range(len(data1_lines)):
            if len(data1_lines[i].split()) == 6:
                data1_lines = data1_lines[i:]
                break

        data1_flat = [float(element) for line in data1_lines for element in line.split()]
        data1_array = np.array(data1_flat)
        data1_xyz = data1_array[:grid_size_x*grid_size_y*grid_size_z].reshape((grid_size_x, grid_size_y, grid_size_z))
        print(data1_xyz.shape)
        fft_data1_xy_shifted = []

        for i in range(grid_size_z):
            data1_xy = data1_xyz[:, :,i]
            fft_data1_xy = fft2(data1_xy)
            fft_data1_xy_shifted_tmp = fftshift(fft_data1_xy)
            fft_data1_xy_shifted.append(fft_data1_xy_shifted_tmp)

        fft_data1_xy_array = np.array(fft_data1_xy_shifted)
        fft_data1_xy_abs = np.abs(fft_data1_xy_array)
        fft_data1_xy_summed = np.sum(fft_data1_xy_abs,axis=0)

    Kx, Ky = np.meshgrid(freq_x, freq_y)
    vmax_value_1 = np.max(fft_data1_xy_summed)

plt.figure(figsize=(20, 8))

im1 = plt.imshow(fft_data1_xy_summed, extent=(Kx.min(), Kx.max(), Ky.min(), Ky.max()), cmap='binary', norm=colors.LogNorm(vmin=1, vmax=vmax_value_1))
plt.xlabel('k(x) / G' if is_periodic == 'yes' else 'k(x)')
plt.ylabel('k(y) / G' if is_periodic == 'yes' else 'k(y)')
plt.rc('grid', linestyle='-', color='black', linewidth=0.5)
plt.grid(True)
cbar1 = plt.colorbar(im1, shrink=0.6) 

plt.rcParams.update({
    'font.family': 'Arial',
    'font.size': 18,
    'axes.edgecolor': 'black',
    'axes.linewidth': 1,
    'axes.grid': True,
    'axes.titlepad': 18,
    'legend.fontsize': 18,
    'figure.edgecolor': 'black',
    'figure.facecolor': 'white',
})

# Define the file path for saving the figure
file_path = input("Enter the file path for saving the figure: ")

# Save the figure at the specified location
plt.savefig(file_path, dpi=300, bbox_inches='tight')


plt.show()